In [ ]:
# Vehicle Trajectory Prediction Using LiDAR and Ego Vehicle Data

This notebook implements a trajectory prediction system that uses vehicle sensor data and LiDAR point clouds to predict the future path of a vehicle.

## Initial Setup and Imports

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import glob
from plyfile import PlyData
from tqdm import tqdm
import math
import pickle
import argparse

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# ## Custom Loss Function

def weighted_displacement_loss(y_true, y_pred):
    """Custom loss function for displacement prediction with temporal weighting"""
    # Extract delta_x and delta_y components
    delta_x_true = y_true[:, :, 0]  # delta_x
    delta_y_true = y_true[:, :, 1]  # delta_y
    
    delta_x_pred = y_pred[:, :, 0]  # delta_x
    delta_y_pred = y_pred[:, :, 1]  # delta_y
    
    # Calculate distance error at each time step
    distance_error = tf.sqrt(tf.square(delta_x_true - delta_x_pred) + tf.square(delta_y_true - delta_y_pred) + 1e-6)
    
    # Weight the errors - more weight on earlier predictions
    time_steps = tf.shape(delta_x_true)[1]
    time_weights = 1.0 / tf.sqrt(tf.cast(tf.range(1, time_steps + 1), tf.float32))
    time_weights = time_weights / tf.reduce_sum(time_weights)  # Normalize
    
    # Apply the weights
    weighted_error = distance_error * tf.expand_dims(time_weights, axis=0)
    
    # Reduce along time dimension
    return tf.reduce_mean(weighted_error)

# ## LiDAR Point Cloud Processing Functions

def load_lidar_point_cloud(file_path):
    """Load LiDAR point cloud from PLY file"""
    try:
        plydata = PlyData.read(file_path)
        x = plydata['vertex']['x']
        y = plydata['vertex']['y']
        z = plydata['vertex']['z']
        intensity = plydata['vertex']['intensity']
        
        # Stack points and intensity
        points = np.column_stack((x, y, z, intensity))
        return points
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return np.zeros((100, 4))  # Return empty array if there's an error

def sample_point_cloud(points, n_points=1024):
    """Sample a fixed number of points from point cloud with improved sampling strategy"""
    if len(points) == 0:
        return np.zeros((n_points, points.shape[1]))
    
    if len(points) >= n_points:
        # Focus sampling on points closer to the vehicle (more relevant for trajectory)
        # Calculate distance from origin (vehicle position)
        distances = np.sqrt(points[:, 0]**2 + points[:, 1]**2)
        
        # Create probability weights - higher probability for closer points
        # Add small constant to avoid division by zero
        weights = 1.0 / (distances + 0.1)
        weights /= np.sum(weights)  # Normalize
        
        # Sample points with weighted probability
        indices = np.random.choice(len(points), n_points, replace=False, p=weights)
        return points[indices]
    else:
        # If we have fewer points than needed, duplicate some points
        indices = np.random.choice(len(points), n_points, replace=True)
        return points[indices]

def process_point_cloud(points, max_points=1024):
    """Process point cloud data with improved normalization"""
    # Extract XYZ coordinates and intensity
    xyz = points[:, :3]
    
    # Center points to origin
    if len(xyz) > 0:
        # Use robust centering - center to median rather than mean
        center = np.median(xyz, axis=0)
        xyz = xyz - center
        
        # Scale to unit sphere using 90th percentile instead of max
        # This makes normalization more robust to outliers
        radii = np.sqrt(np.sum(xyz**2, axis=1))
        if len(radii) > 0:
            scale = np.percentile(radii, 90)
            if scale > 0:
                xyz = xyz / scale
    
    # Include intensity as an additional feature, normalized to [0,1]
    if points.shape[1] > 3:
        intensity = points[:, 3].reshape(-1, 1)
        if intensity.size > 0:
            intensity_max = np.max(intensity) if np.max(intensity) > 0 else 1.0
            intensity = intensity / intensity_max
        processed_points = np.hstack((xyz, intensity))
    else:
        processed_points = xyz
    
    return processed_points

# ## Data Processing Functions

def map_timestamps_to_lidar(ego_df, lidar_path):
    """Map timestamps from ego data to LiDAR files with improved matching"""
    lidar_files = glob.glob(os.path.join(lidar_path, 'lidar_*.ply'))
    lidar_timestamps = [float(os.path.splitext(os.path.basename(f))[0].split('_')[1]) for f in lidar_files]
    
    # Sort timestamps for more efficient searching
    lidar_timestamps = np.array(lidar_timestamps)
    sorted_indices = np.argsort(lidar_timestamps)
    lidar_timestamps = lidar_timestamps[sorted_indices]
    lidar_files = [lidar_files[i] for i in sorted_indices]
    
    # Create mapping from ego timestamps to closest LiDAR timestamp
    timestamp_to_lidar = {}
    
    for idx, ts in enumerate(ego_df['timestamp']):
        # Find closest LiDAR timestamp using binary search
        if len(lidar_timestamps) > 0:
            closest_idx = np.searchsorted(lidar_timestamps, ts)
            
            # Handle edge cases
            if closest_idx == 0:
                closest_match = closest_idx
            elif closest_idx == len(lidar_timestamps):
                closest_match = closest_idx - 1
            else:
                # Compare distances to find the closest timestamp
                if abs(lidar_timestamps[closest_idx] - ts) < abs(lidar_timestamps[closest_idx-1] - ts):
                    closest_match = closest_idx
                else:
                    closest_match = closest_idx - 1
                
            timestamp_to_lidar[ts] = lidar_files[closest_match]
    
    return timestamp_to_lidar

def calculate_vehicle_dynamics(ego_df):
    """Calculate enhanced vehicle dynamics features"""
    # Copy dataframe to avoid modifying the original
    df = ego_df.copy()
    
    # Calculate speed from velocity components
    df['speed'] = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2)
    
    # Calculate heading in radians (-π to π)
    df['heading'] = np.arctan2(df['velocity_y'], df['velocity_x'])
    
    # Calculate acceleration components and magnitude
    df['acceleration_x'] = df['velocity_x'].diff() / df['timestamp'].diff()
    df['acceleration_y'] = df['velocity_y'].diff() / df['timestamp'].diff()
    df['acceleration'] = np.sqrt(df['acceleration_x']**2 + df['acceleration_y']**2)
    
    # Calculate jerk (rate of change of acceleration)
    df['jerk_x'] = df['acceleration_x'].diff() / df['timestamp'].diff()
    df['jerk_y'] = df['acceleration_y'].diff() / df['timestamp'].diff()
    df['jerk'] = np.sqrt(df['jerk_x']**2 + df['jerk_y']**2)
    
    # Calculate heading change rate (angular velocity)
    df['heading_change'] = df['heading'].diff() / df['timestamp'].diff()
    
    # Calculate curvature (rate of change of heading with respect to distance)
    # Avoid division by zero by adding a small constant
    df['curvature'] = df['heading_change'] / (df['speed'] + 1e-6)
    
    # Identify whether the vehicle is effectively stationary
    df['is_moving'] = (df['speed'] > 0.5).astype(float)
    
    # Fill NaN values
    for col in df.columns:
        if df[col].dtype in [np.float64, np.float32]:
            df[col] = df[col].fillna(0)
    
    return df

def calculate_relative_displacement(ego_df):
    """Calculate relative displacements instead of absolute positions"""
    df = ego_df.copy()
    
    # Calculate displacement (delta) between consecutive frames
    df['delta_x'] = df['x'].diff()
    df['delta_y'] = df['y'].diff()
    
    # Fill NaN values with 0 (no movement)
    df['delta_x'] = df['delta_x'].fillna(0)
    df['delta_y'] = df['delta_y'].fillna(0)
    
    return df

def filter_and_segment_data(ego_df, min_speed=0.5, min_segment_length=5):
    """Filter stationary data and segment into meaningful driving sequences"""
    df = ego_df.copy()
    
    # Mark segments with speed above threshold
    df['is_moving'] = (df['speed'] > min_speed).astype(int)
    df['movement_change'] = df['is_moving'].diff().abs()
    df.loc[0, 'movement_change'] = 0  # Fix first row
    
    # Create segment IDs
    df['segment_id'] = df['movement_change'].cumsum()
    
    # Get segment lengths
    segment_lengths = df.groupby('segment_id').size()
    valid_segments = segment_lengths[segment_lengths >= min_segment_length].index
    
    # Filter to only include valid segments
    df_filtered = df[df['segment_id'].isin(valid_segments)].copy()
    
    # Reset segment_ids to be consecutive
    segment_mapping = {old_id: new_id for new_id, old_id in enumerate(df_filtered['segment_id'].unique())}
    df_filtered['segment_id'] = df_filtered['segment_id'].map(segment_mapping)
    
    # Create a binary column indicating if this is the first frame of a segment
    df_filtered['segment_start'] = df_filtered['segment_id'].diff().ne(0).astype(int)
    df_filtered.loc[df_filtered.index[0], 'segment_start'] = 1  # First row is always start of segment
    
    return df_filtered

# ## Sequence Preparation Functions

def prepare_single_sequence(df_filtered, timestamp_to_lidar, current_idx, seq_length=10, 
                           scaler_input=None, input_features=None):
    """
    Prepare a single sequence for prediction using relative displacement representation
    
    Args:
        df_filtered: Processed ego vehicle data
        timestamp_to_lidar: Mapping of timestamps to LiDAR files
        current_idx: Index in the dataframe to use as the current position
        seq_length: Length of the sequence to use as input
        scaler_input: Trained scaler for input normalization
        input_features: List of input features to use
    
    Returns:
        sequence_input: Normalized sequence data
        lidar_input: Processed LiDAR point cloud
        start_position: (x, y) coordinates of the current position
        start_heading: Current heading in radians
    """
    if input_features is None:
        input_features = [
            'delta_x', 'delta_y',
            'velocity_x', 'velocity_y', 'speed',
            'heading', 'heading_change', 'curvature',
            'acceleration_x', 'acceleration_y', 'acceleration',
            'steering', 'throttle', 'brake',
            'is_moving'
        ]
    
    # Make sure we have enough history
    if current_idx < seq_length:
        print(f"Warning: Not enough history at index {current_idx}. Need at least {seq_length} frames.")
        return None, None, None, None
    
    # Extract the sequence
    seq_indices = range(current_idx - seq_length, current_idx)
    input_seq = df_filtered.loc[seq_indices, input_features].values
    
    # Normalize the sequence
    if scaler_input is not None:
        input_seq_norm = scaler_input.transform(input_seq)
    else:
        input_seq_norm = input_seq
        
    # Get current position and heading
    start_x = df_filtered.loc[current_idx - 1, 'x']
    start_y = df_filtered.loc[current_idx - 1, 'y']
    start_heading = df_filtered.loc[current_idx - 1, 'heading']
    
    # Get LiDAR data for the current timestep
    current_ts = df_filtered.loc[current_idx - 1, 'timestamp']
    if current_ts in timestamp_to_lidar:
        lidar_file = timestamp_to_lidar[current_ts]
        pointcloud = load_lidar_point_cloud(lidar_file)
        sampled_points = sample_point_cloud(pointcloud)
        processed_points = process_point_cloud(sampled_points)
    else:
        print(f"Warning: No LiDAR data found for timestamp {current_ts}")
        processed_points = np.zeros((1024, 4))
    
    return input_seq_norm, processed_points, (start_x, start_y), start_heading

# ## Trajectory Prediction Functions

def convert_relative_to_absolute(start_x, start_y, relative_displacements):
    """Convert relative displacements to absolute positions"""
    absolute_positions = np.zeros((len(relative_displacements) + 1, 2))
    absolute_positions[0] = [start_x, start_y]
    
    for i in range(len(relative_displacements)):
        absolute_positions[i+1, 0] = absolute_positions[i, 0] + relative_displacements[i, 0]
        absolute_positions[i+1, 1] = absolute_positions[i, 1] + relative_displacements[i, 1]
    
    return absolute_positions

def calculate_steering_from_path(future_positions, wheelbase=2.7):
    """Calculate steering angles from a predicted path using bicycle model"""
    steering_angles = []
    
    for i in range(1, len(future_positions)):
        # Calculate the displacement vector
        dx = future_positions[i][0] - future_positions[i-1][0]
        dy = future_positions[i][1] - future_positions[i-1][1]
        
        # Calculate the heading change
        if i == 1:
            # For the first point, assume the vehicle is already pointing in the direction of travel
            heading_change = 0
        else:
            prev_dx = future_positions[i-1][0] - future_positions[i-2][0]
            prev_dy = future_positions[i-1][1] - future_positions[i-2][1]
            
            # Calculate headings
            prev_heading = math.atan2(prev_dy, prev_dx)
            current_heading = math.atan2(dy, dx)
            
            # Calculate heading change (handle wrap-around)
            heading_change = current_heading - prev_heading
            if heading_change > math.pi:
                heading_change -= 2 * math.pi
            elif heading_change < -math.pi:
                heading_change += 2 * math.pi
        
        # Calculate path radius using the bicycle model formula
        # For small displacements, use a large radius to avoid extreme steering
        displacement = math.sqrt(dx**2 + dy**2)
        if abs(heading_change) < 1e-6 or displacement < 1e-3:
            radius = float('inf')  # Straight line
        else:
            radius = displacement / heading_change  # Approximate radius of curvature
        
        # Calculate steering angle using the bicycle model (Ackermann steering)
        if radius == float('inf'):
            steering_angle = 0
        else:
            steering_angle = math.atan(wheelbase / radius)
        
        steering_angles.append(steering_angle)
    
    # For the first point, use the steering calculated for the second point
    if steering_angles:
        steering_angles.insert(0, steering_angles[0])
    else:
        steering_angles.append(0)  # Default to zero if we don't have enough points
    
    return steering_angles

def predict_trajectory(model, sequence_input, lidar_input, start_position, scaler_target=None):
    """
    Predict a future trajectory given current state
    
    Args:
        model: Trained trajectory prediction model
        sequence_input: Normalized sequence of recent vehicle dynamics
        lidar_input: Processed LiDAR point cloud data
        start_position: Tuple of (x, y) for current position
        scaler_target: Scaler for target normalization
    
    Returns:
        predicted_path: Array of absolute positions for the predicted path
        steering_angles: Array of steering angles corresponding to the path
    """
    # Expand dimensions for batch size
    sequence_input = np.expand_dims(sequence_input, axis=0)
    lidar_input = np.expand_dims(lidar_input, axis=0)
    
    # Get model prediction
    prediction = model.predict([sequence_input, lidar_input])
    
    # Reshape prediction for inverse transformation
    pred_reshaped = prediction.reshape(-1, 2)
    
    # Inverse transform to get real-world displacements
    if scaler_target is not None:
        pred_original = scaler_target.inverse_transform(pred_reshaped)
    else:
        pred_original = pred_reshaped
    
    # Reshape back
    pred_original = pred_original.reshape(prediction.shape)[0]
    
    # Convert relative displacements to absolute positions
    start_x, start_y = start_position
    predicted_path = convert_relative_to_absolute(start_x, start_y, pred_original)
    
    # Calculate steering angles from the path
    steering_angles = calculate_steering_from_path(predicted_path)
    
    return predicted_path, steering_angles

# ## Visualization Functions

def visualize_prediction(predicted_path, start_position, start_heading, save_path=None):
    """
    Visualize a predicted trajectory
    
    Args:
        predicted_path: Array of predicted positions
        start_position: Tuple of (x, y) for starting position
        start_heading: Starting heading in radians
        save_path: Path to save the figure (if None, the figure is displayed)
    """
    plt.figure(figsize=(10, 8))
    
    # Plot the predicted path
    plt.plot(predicted_path[:, 0], predicted_path[:, 1], 'r-', linewidth=2, label='Predicted Path')
    
    # Mark starting position
    start_x, start_y = start_position
    plt.scatter(start_x, start_y, color='green', s=100, zorder=5, label='Current Position')
    
    # Draw an arrow indicating the starting heading
    arrow_length = 2.0  # meters
    dx = arrow_length * np.cos(start_heading)
    dy = arrow_length * np.sin(start_heading)
    plt.arrow(start_x, start_y, dx, dy, head_width=0.5, head_length=0.7, 
              fc='green', ec='green', zorder=5)
    
    # Mark points along the path
    for i in range(1, len(predicted_path), 2):
        plt.scatter(predicted_path[i, 0], predicted_path[i, 1], color='blue', s=30, alpha=0.7)
    
    # Add arrows to show direction
    for i in range(1, len(predicted_path)-1, 2):
        dx = predicted_path[i+1, 0] - predicted_path[i, 0]
        dy = predicted_path[i+1, 1] - predicted_path[i, 1]
        plt.arrow(predicted_path[i, 0], predicted_path[i, 1], dx*0.5, dy*0.5, 
                  head_width=0.3, head_length=0.5, fc='blue', ec='blue', alpha=0.5)
    
    # Set equal aspect ratio
    plt.axis('equal')
    plt.grid(True)
    plt.title('Predicted Vehicle Trajectory')
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.legend()
    
    # Add distance markers
    total_distance = 0
    for i in range(1, len(predicted_path)):
        dx = predicted_path[i, 0] - predicted_path[i-1, 0]
        dy = predicted_path[i, 1] - predicted_path[i-1, 1]
        segment_distance = np.sqrt(dx**2 + dy**2)
        total_distance += segment_distance
    
    plt.figtext(0.02, 0.02, f'Total path distance: {total_distance:.2f} m', fontsize=10)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Visualization saved to {save_path}")
    else:
        plt.show()

def predict_multiple_steps(model, ego_df, timestamp_to_lidar, start_idx, num_steps, 
                         seq_length, scaler_input, scaler_target, input_features, visualize=True):
    """
    Predict trajectory for multiple consecutive steps, using previous predictions as input
    for subsequent predictions
    
    Args:
        model: Trained trajectory prediction model
        ego_df: Processed ego vehicle data
        timestamp_to_lidar: Mapping of timestamps to LiDAR files
        start_idx: Index to start predictions from
        num_steps: Number of consecutive prediction steps
        seq_length: Length of input sequence for model
        scaler_input: Scaler for input normalization
        scaler_target: Scaler for target normalization
        input_features: List of input features
        visualize: Whether to visualize each prediction
    
    Returns:
        List of predicted paths
    """
    predictions = []
    current_idx = start_idx
    
    for step in range(num_steps):
        print(f"Predicting step {step+1}/{num_steps}")
        
        # Prepare sequence for current position
        seq_input, lidar_input, start_position, start_heading = prepare_single_sequence(
            ego_df, timestamp_to_lidar, current_idx, seq_length, scaler_input, input_features
        )
        
        if seq_input is None:
            print(f"Cannot predict at step {step+1}. Stopping.")
            break
        
        # Predict trajectory
        predicted_path, steering_angles = predict_trajectory(
            model, seq_input, lidar_input, start_position, scaler_target
        )
        
        predictions.append({
            'path': predicted_path,
            'steering': steering_angles,
            'start_position': start_position,
            'start_heading': start_heading
        })
        
        if visualize:
            save_path = f"prediction_step_{step+1}.png"
            visualize_prediction(predicted_path, start_position, start_heading, save_path)
        
        # Move to next position
        current_idx += 1
    
    return predictions

# ## Running Predictions

# Example usage - load and run model on data
def run_trajectory_prediction(data_path, model_path, scalers_path, seq_length=10, 
                             prediction_idx=None, num_predictions=1, visualize=True):
    """
    Run trajectory prediction on new data
    
    Args:
        data_path: Path to the directory containing ego data and LiDAR files
        model_path: Path to the saved model file
        scalers_path: Path to the saved scalers file
        seq_length: Length of input sequence for model
        prediction_idx: Specific index to predict (if None, will find a good segment)
        num_predictions: Number of consecutive predictions to make
        visualize: Whether to visualize predictions
    """
    # Load model with custom loss function
    print(f"Loading model from {model_path}")
    model = keras.models.load_model(model_path, 
                                   custom_objects={'weighted_displacement_loss': weighted_displacement_loss})
    
    # Load scalers
    print(f"Loading scalers from {scalers_path}")
    with open(scalers_path, 'rb') as f:
        scalers = pickle.load(f)
        scaler_input = scalers['input_scaler']
        scaler_target = scalers['target_scaler']
    
    # Load and process data
    ego_data_file = glob.glob(os.path.join(data_path, 'ego_data_*.csv'))[0]
    lidar_path = os.path.join(data_path, 'lidar')
    
    print(f"Loading ego data from: {ego_data_file}")
    print(f"Loading LiDAR data from: {lidar_path}")
    
    # Load ego vehicle data
    ego_df = pd.read_csv(ego_data_file)
    print(f"Loaded ego data with {len(ego_df)} timesteps")
    
    # Process ego data
    print("Processing ego vehicle data...")
    ego_df_dynamics = calculate_vehicle_dynamics(ego_df)
    ego_df_relative = calculate_relative_displacement(ego_df_dynamics)
    ego_df_filtered = filter_and_segment_data(ego_df_relative)
    
    print(f"Filtered data contains {len(ego_df_filtered)} timesteps in "
          f"{ego_df_filtered['segment_id'].nunique()} driving segments")
    
    # Map timestamps to LiDAR files
    print("Mapping timestamps to LiDAR files...")
    timestamp_to_lidar = map_timestamps_to_lidar(ego_df_filtered, lidar_path)
    print(f"Matched {len(timestamp_to_lidar)} timestamps to LiDAR files")
    
    # Define input features
    input_features = [
        'delta_x', 'delta_y',
        'velocity_x', 'velocity_y', 'speed',
        'heading', 'heading_change', 'curvature',
        'acceleration_x', 'acceleration_y', 'acceleration',
        'steering', 'throttle', 'brake',
        'is_moving'
    ]
    
    # Choose prediction index
    if prediction_idx is not None:
        start_idx = prediction_idx
    else:
        # Find segments with good speed
        segments = ego_df_filtered.groupby('segment_id')
        good_segments = []
        
        for segment_id, segment_data in segments:
            if len(segment_data) >= seq_length + 10 and segment_data['speed'].mean() > 3.0:
                good_segments.append({
                    'segment_id': segment_id,
                    'start_idx': segment_data.index[seq_length],
                    'avg_speed': segment_data['speed'].mean(),
                    'length': len(segment_data)
                })
        
        if good_segments:
            # Sort by average speed (descending)
            good_segments.sort(key=lambda x: x['avg_speed'], reverse=True)
            best_segment = good_segments[0]
            start_idx = best_segment['start_idx']
            print(f"Selected segment {best_segment['segment_id']} with avg speed {best_segment['avg_speed']:.2f} m/s")
        else:
            # If no good segments found, use the middle of the first segment
            first_segment = ego_df_filtered[ego_df_filtered['segment_id'] == 0]
            start_idx = first_segment.index[min(seq_length, len(first_segment)-1)]
            print(f"No good segments found, using index {start_idx}")
    
    print(f"Starting predictions from index {start_idx}")
    
    # Make predictions
    predictions = predict_multiple_steps(
        model, ego_df_filtered, timestamp_to_lidar, 
        start_idx, num_predictions, 
        seq_length, scaler_input, scaler_target, 
        input_features, visualize
    )
    
    # Print prediction results
    for i, pred in enumerate(predictions):
        print(f"\nPrediction {i+1}:")
        start_x, start_y = pred['start_position']
        print(f"  Starting position: ({start_x:.2f}, {start_y:.2f})")
        print(f"  Starting heading: {math.degrees(pred['start_heading']):.2f} degrees")
        print(f"  First steering angle: {math.degrees(pred['steering'][0]):.2f} degrees")
        print(f"  Last steering angle: {math.degrees(pred['steering'][-1]):.2f} degrees")
    
    return predictions

# ## Example Usage with Parameters

# This is an example of how you might run the code in a notebook environment
# In a real notebook, you would set these parameters and run the cell
'''
# Set paths and parameters
data_path = '/path/to/data'
model_path = 'trajectory_prediction_model.h5'
scalers_path = 'trajectory_scalers.pkl'
seq_length = 10
num_predictions = 3
visualize = True

# Run trajectory prediction
predictions = run_trajectory_prediction(
    data_path=data_path,
    model_path=model_path,
    scalers_path=scalers_path,
    seq_length=seq_length,
    prediction_idx=None,  # Auto-select a good segment
    num_predictions=num_predictions,
    visualize=visualize
)

# Analyze results
for i, pred in enumerate(predictions):
    # Plot the predicted path
    plt.figure(figsize=(10, 8))
    path = pred['path']
    plt.plot(path[:, 0], path[:, 1], 'b-', linewidth=2)
    plt.scatter(path[0, 0], path[0, 1], color='green', s=100)
    plt.axis('equal')
    plt.grid(True)
    plt.title(f'Predicted Path {i+1}')
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.show()
'''